In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 500)

from sklearn import svm
import re
from shutil import copyfile
import matplotlib.pyplot as plt
import pydicom
from tqdm import tqdm
import nibabel as nib
import dicom2nifti

In [2]:
# define text files and path

#### CHANGE FOR SPECIFIC DOWNLOAD PACKAGE ####

# for 2274:
path = '../../physcosis/Package_1187332/'


## for 2126:
# path = '../../physcosis/Package_1187335/'


# should be standard
test_file = 'panss01.txt'
subject_file = 'ndar_subject01.txt'
image_file = 'image03.txt'

In [3]:
image03 = pd.read_csv(os.path.join(path, image_file), delimiter = '\t')
print(image03.shape)

(9395, 51)


In [4]:
# image03[image03['image_description'] == 'ADNI Double_TSE SENSE']
# image03

In [5]:
data_dir = os.path.join(path, 'image03')

#### CHANGE BASED ON WHAT 'image_file' COLUMN LOOKS LIKE IN image03 (corrects the file path to be on local machine) ####

# for 2274:
regex ='s3://NDAR_.*/submission_.[0-9]*/'


rows_with_bad_path = []
for index, row in image03.iterrows():
    image_path = re.sub(regex, '', row['image_file'])
    full_path = os.path.join(data_dir, image_path)
#     print(full_path)
    if not os.path.exists(full_path):
        print(image_path, index, row['image_file'])
        rows_with_bad_path.append(index)
rows_with_bad_path

Data file (image, behavioral, anatomical, etc) 0 Data file (image, behavioral, anatomical, etc)


[0]

In [6]:
# drops rows based on rows with bad path and replaces old paths with local paths

image03 = image03.drop(rows_with_bad_path)
l = []

for index, row in image03.iterrows():
    image_path = re.sub(regex, '', row['image_file'])
    full_path = os.path.join(data_dir, image_path)
    l.append(full_path)
image03['image_file'] = l


In [7]:

# shows the different types of images available for the data package
pivot_table = image03.pivot_table(columns = ['image_description'], aggfunc = 'size')
pivot_table

image_description
ADNI Double_TSE SENSE                      561
ADNI MPRAGE SENSE                           45
ADNI MPRAGE pg7 SENSE                      516
AX T2                                       16
Axial PD-T2 TSE                            118
Axial PD-T2 TSE no angle                    73
Axial PD-T2 TSE_ NO ANGULATION               4
Axial PD-T2 TSE_NO_ANGLE                    26
B1-Calibration Body                         98
B1-Calibration Body_COIL                    25
B1-Callibration Head                        98
B1-Callibration Head_COIL                   24
Bsnip Ax PD-T2 TSE NoAng                   290
Bsnip Rest 210                             244
Bsnip Sag MPRAGE-like ADNI                 291
Bsnip ep2d_diff_MGH-DTI2                   755
Bsnip ep2d_diff_MGH-DTI2_split_1             1
DTI                                         55
DTI-exact repeat of DTI-new                271
DTI-exact repeat of DTI-new_ADC            270
DTI-exact repeat of DTI-new_ColFA         

In [8]:
len(image03['image_description'].unique())

76

In [9]:
test_dir = os.path.join(data_dir, 'TEST')

if not os.path.exists(test_dir):
    os.mkdir(test_dir)
test_dir

'../../physcosis/Package_1187332/image03/TEST'

In [10]:
image03 = image03.drop_duplicates(subset = ['image_description'], keep='first')
image03.shape

(76, 51)

In [13]:
for i,row in image03.iterrows():
    file_name = row['image_file'].split('/')[-1]
    
    folder = os.path.join(test_dir, file_name.split('.')[0])
    if not os.path.exists(folder):
        os.system('mkdir ' + folder)
        os.system('unzip ' + row['image_file'] + ' -d ' + folder)
        print('Unziped and moved ', file_name)

Unziped and moved  S1551MJW-1-10.zip
Unziped and moved  S3715XQE-1-7.zip
Unziped and moved  S5565YKS-1-16.zip
Unziped and moved  S7657KXI-1-4.zip
Unziped and moved  S4164MBS-1-3.zip
Unziped and moved  S6705CRG-1-5.zip
Unziped and moved  S1549PLY-1-5.zip
Unziped and moved  S6705CRG-1-3.zip
Unziped and moved  S1549PLY-1-3.zip
Unziped and moved  S1549PLY-1-4.zip
Unziped and moved  S9263VVF-1-5.zip
Unziped and moved  S7765ICN-1-1.zip
Unziped and moved  S7765ICN-1-2.zip
Unziped and moved  S7765ICN-1-4.zip
Unziped and moved  S7765ICN-1-5.zip
Unziped and moved  S7765ICN-1-6.zip
Unziped and moved  S7696EHV-1-6.zip
Unziped and moved  S6506IPR-1-4.zip
Unziped and moved  S6506IPR-1-5.zip
Unziped and moved  S3185NLP-1-7.zip
Unziped and moved  S0502LWN-1-2.zip
Unziped and moved  S9860SVG-1-3.zip
Unziped and moved  S4673PTE-1-6.zip
Unziped and moved  S3772DPX-1-6.zip
Unziped and moved  S3772DPX-1-11.zip
Unziped and moved  S4524DRI-1-21.zip
Unziped and moved  S9061TUS-1-6.zip
Unziped and moved  S9136

In [36]:
new_path = []
for i,row in image03.iterrows():
    folder = row['image_file'].split('/')[-1].split('.')[0]
    p = os.path.join(test_dir, folder)
    nifti = os.path.join(p, '{}.nii'.format(folder))
    if not os.path.exists(nifti):
        print('Converting ', folder, ' from dicom to niftii.')
        try:
            dicom2nifti.dicom_series_to_nifti(p, nifti, reorient_nifti=True)
            new_path.append(nifti)
        except: 
            new_path.append('FAILED')
            print(folder, ' Failed.')
        
    else: 
        new_path.append(nifti)
    


Converting  S1551MJW-1-10  from dicom to niftii.
S1551MJW-1-10  Failed.
Converting  S3715XQE-1-7  from dicom to niftii.
S3715XQE-1-7  Failed.
Converting  S5565YKS-1-16  from dicom to niftii.
Converting  S7657KXI-1-4  from dicom to niftii.
S7657KXI-1-4  Failed.
Converting  S4164MBS-1-3  from dicom to niftii.
S4164MBS-1-3  Failed.
Converting  S6705CRG-1-5  from dicom to niftii.
S6705CRG-1-5  Failed.
Converting  S1549PLY-1-5  from dicom to niftii.
S1549PLY-1-5  Failed.
Converting  S6705CRG-1-3  from dicom to niftii.
S6705CRG-1-3  Failed.
Converting  S1549PLY-1-3  from dicom to niftii.
S1549PLY-1-3  Failed.
Converting  S1549PLY-1-4  from dicom to niftii.
S1549PLY-1-4  Failed.
Converting  S9263VVF-1-5  from dicom to niftii.
S9263VVF-1-5  Failed.
Converting  S7765ICN-1-1  from dicom to niftii.


Missing slices (slice count mismatch between timepoint 0 and 1)
---------------------------------------------------------
(256, 256, 4)
(256, 256, 5)
---------------------------------------------------------


S7765ICN-1-1  Failed.
Converting  S7765ICN-1-2  from dicom to niftii.


Slice increment not consistent through all slices
---------------------------------------------------------
[  30.6881 -145.287   110.751 ] [-1.2  0.   0. ]
[  33.0881 -145.287   110.751 ] [-2.4  0.   0. ]
Instance Number: 56
---------------------------------------------------------


S7765ICN-1-2  Failed.
Converting  S7765ICN-1-4  from dicom to niftii.


...

S8162MMI-1-8  Failed.
Converting  S8162MMI-1-9  from dicom to niftii.


Slice increment not consistent through all slices
---------------------------------------------------------
[ -42.28510857 -149.30459464  132.13452721] [-2.5  0.   0. ]
[ -37.28510857 -149.30459464  132.13452721] [-5.  0.  0.]
Instance Number: 39
---------------------------------------------------------


S8162MMI-1-9  Failed.
Converting  S2147OET-1-11  from dicom to niftii.
S2147OET-1-11  Failed.
Converting  S4051NAD-1-3  from dicom to niftii.
Converting  S4051NAD-1-11  from dicom to niftii.
S4051NAD-1-11  Failed.
Converting  S0483IXU-1-4  from dicom to niftii.
Converting  S0483IXU-1-5  from dicom to niftii.
Converting  S0483IXU-1-8  from dicom to niftii.
S0483IXU-1-8  Failed.
Converting  S8827HHF-1-6  from dicom to niftii.
Converting  S4303SMC-1-13  from dicom to niftii.
Converting  S2707LVO-1-9  from dicom to niftii.
Converting  S2707LVO-1-10  from dicom to niftii.


Slice increment not consistent through all slices
---------------------------------------------------------
[  14.84406776 -140.10362625  120.06188822] [-1.2  0.   0. ]
[  17.24406776 -140.10362625  120.06188822] [-2.4  0.   0. ]
Instance Number: 96
---------------------------------------------------------


S2707LVO-1-10  Failed.
Converting  S4693KFE-1-3  from dicom to niftii.
Converting  S4229VAB-1-2  from dicom to niftii.
S4229VAB-1-2  Failed.
Converting  S9061TUS-1-5  from dicom to niftii.
Converting  S3177QQP-1-16  from dicom to niftii.
Converting  S3500OTV-1-4  from dicom to niftii.
Converting  S0783EEY-1-5  from dicom to niftii.
Converting  S8339DPJ-1-3  from dicom to niftii.
Converting  S2911XGM-1-2  from dicom to niftii.
Converting  S6572CVP-1-2  from dicom to niftii.


In [38]:
image03['newpath'] = new_path

In [40]:
image03 = image03[image03.newpath != 'FAILED']

In [70]:
image03 = image03.reset_index()
image03

,level_0,index,collection_id,image03_id,dataset_id,subjectkey,src_subject_id,interview_age,sex,comments_misc,...,image_slice_thickness,image_orientation,pulse_seq,slice_acquisition,week,experiment_description,slice_timing,bvek_bval_files,collection_title,newpath
0,0,802,2274,462790,11322,NDAR_INVEL57K1YW,960,312,F,ORIGINALPRIMARYMND,...,3,Axial,ep_b1000#5,1,0,Bsnip ep2d_diff_MGH-DTI2,Yes,Yes,Bipolar & Schizophrenia Consortium for Parsing Intermediate Phenotypes (B-SNIP 1),../../physcosis/Package_1187332/image03/TEST/S5565YKS-1-16/S5565YKS-1-16.nii
1,1,2353,2274,455307,11148,NDAR_INVP8TKTHBT,2395,276,F,ORIGINALPRIMARYMND,...,1.2,Axial,*tfl3d1_ns,1,0,T1 MPRAGE,Yes,NaN,Bipolar & Schizophrenia Consortium for Parsing Intermediate Phenotypes (B-SNIP 1),../../physcosis/Package_1187332/image03/TEST/S0502LWN-1-2/S0502LWN-1-2.nii
2,2,2459,2274,455109,11148,NDAR_INVHWTD7MDH,2103,588,M,ORIGINALPRIMARYMND,...,1.2,Axial,*tfl3d1_ns,1,0,T1 MPRAGE Repeat,Yes,NaN,Bipolar & Schizophrenia Consortium for Parsing Intermediate Phenotypes (B-SNIP 1),../../physcosis/Package_1187332/image03/TEST/S9860SVG-1-3/S9860SVG-1-3.nii
3,3,2697,2274,458432,11161,NDAR_INVTBF7RLNK,3944,324,M,ORIGINALPRIMARYMNDNORMMOSAIC,...,3,Axial,*ep_b1000#2,1,0,DTI,Yes,Yes,Bipolar & Schizophrenia Consortium for Parsing Intermediate Phenotypes (B-SNIP 1),../../physcosis/Package_1187332/image03/TEST/S4673PTE-1-6/S4673PTE-1-6.nii
4,4,2698,2274,458433,11161,NDAR_INV7U2A6KF9,2334,264,F,ORIGINALPRIMARYMNDMOSAIC,...,3,Axial,*ep_b1000#16,1,0,DTI-new-12.8.08,Yes,Yes,Bipolar & Schizophrenia Consortium for Parsing Intermediate Phenotypes (B-SNIP 1),../../physcosis/Package_1187332/image03/TEST/S3772DPX-1-6/S3772DPX-1-6.nii
5,5,2699,2274,458434,11161,NDAR_INV7U2A6KF9,2334,264,F,ORIGINALPRIMARYMNDMOSAIC,...,3,Axial,*ep_b1000#28,1,0,DTI-exact repeat of DTI-new,Yes,Yes,Bipolar & Schizophrenia Consortium for Parsing Intermediate Phenotypes (B-SNIP 1),../../physcosis/Package_1187332/image03/TEST/S3772DPX-1-11/S3772DPX-1-11.nii
6,6,2795,2274,458648,11161,NDAR_INVVC9DLH8U,3326,504,F,ORIGINALPRIMARYMNDMOSAIC,...,3,Axial,*ep_b1000#28,1,0,ep2d_diff_convert_Allegra_B-SNIP,Yes,Yes,Bipolar & Schizophrenia Consortium for Parsing Intermediate Phenotypes (B-SNIP 1),../../physcosis/Package_1187332/image03/TEST/S4524DRI-1-21/S4524DRI-1-21.nii
7,7,2851,2274,458467,11161,NDAR_INVAPHKLMDP,2215,696,F,ORIGINALPRIMARYMNDNORMMOSAIC,...,3,Axial,*ep_b1000#2,1,0,DTI-original,Yes,Yes,Bipolar & Schizophrenia Consortium for Parsing Intermediate Phenotypes (B-SNIP 1),../../physcosis/Package_1187332/image03/TEST/S9061TUS-1-6/S9061TUS-1-6.nii
8,8,3038,2274,456192,11161,NDAR_INVBMGW8MDK,2980,516,F,ORIGINALPRIMARYADCND,...,3,Axial,*ep_b0_1000,1,0,DTI-exact repeat of DTI-new,Yes,Yes,Bipolar & Schizophrenia Consortium for Parsing Intermediate Phenotypes (B-SNIP 1),../../physcosis/Package_1187332/image03/TEST/S9080BQA-1-12/S9080BQA-1-12.nii
9,9,3039,2274,456193,11161,NDAR_INVBMGW8MDK,2980,516,F,ORIGINALPRIMARYDIFFUSIONTRACEWND,...,3,Axial,*ep_b1000t,1,0,DTI-exact repeat of DTI-new,Yes,Yes,Bipolar & Schizophrenia Consortium for Parsing Intermediate Phenotypes (B-SNIP 1),../../physcosis/Package_1187332/image03/TEST/S9080BQA-1-13/S9080BQA-1-13.nii


In [109]:
index = 18
path = image03.iloc[index]['newpath']
print(image03.iloc[index]['image_description'])
img = nib.load(path)
img = img.get_fdata()
img.shape

ep2d_diff_convert_Allegra_B-SNIP_ADC


(128, 128, 45)

In [110]:
img_t1 = img[:,:,:]
import imageio

In [111]:
# img_t1 = img[:,:,:,0]
num_img = len(list(range(0,img_t1.shape[2],1)))
fig = plt.figure(figsize=(10, 2), dpi=100)
images = []
for i,p in enumerate(range(0,img_t1.shape[2],1)):
    images.append(img_t1[:,:,p])
    
imageio.mimsave('./movie.gif', images)

    
    
    
    
#     ax1 = fig.add_subplot(1,num_img,i+1)
#     plt.imshow(img_t1[p,:,:], cmap = 'gray')
    
# num_img = len(list(range(0,img.shape[1],40)))
# fig = plt.figure(figsize=(20, 2), dpi=100)
# for i,p in enumerate(range(0,img.shape[0],40)):
#     ax1 = fig.add_subplot(1,num_img,i+1)
#     plt.imshow(img_t1[:,p,:], cmap = 'gray')
    
# num_img = len(list(range(0,img.shape[2],10)))
# fig = plt.figure(figsize=(20, 2), dpi=100)
# for i,p in enumerate(range(0,img.shape[0],10)):
#     ax1 = fig.add_subplot(1,num_img,i+1)
#     plt.imshow(img_t1[:,:,p], cmap = 'gray')

Lossy conversion from float64 to uint8. Range [0.0, 3628.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3718.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3533.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3536.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3773.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3650.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3448.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3633.0]. Convert image to uint8 prior to saving to suppress this w

<Figure size 1000x200 with 0 Axes>

In [112]:
image03['image_description']

0            Bsnip ep2d_diff_MGH-DTI2_split_1
1                                   T1 MPRAGE
2                            T1 MPRAGE Repeat
3                                         DTI
4                             DTI-new-12.8.08
5                 DTI-exact repeat of DTI-new
6            ep2d_diff_convert_Allegra_B-SNIP
7                                DTI-original
8             DTI-exact repeat of DTI-new_ADC
9          DTI-exact repeat of DTI-new_TRACEW
10             DTI-exact repeat of DTI-new_FA
11                        DTI-new-12.8.08_ADC
12                     DTI-new-12.8.08_TRACEW
13                         DTI-new-12.8.08_FA
14                                    DTI_ADC
15                                 DTI_TRACEW
16                           DTI-original_ADC
17                        DTI-original_TRACEW
18       ep2d_diff_convert_Allegra_B-SNIP_ADC
19    ep2d_diff_convert_Allegra_B-SNIP_TRACEW
20        ep2d_diff_convert_Allegra_B-SNIP_FA
21                              MP